In [0]:
%run "./config"

In [0]:
spark.conf.set(
    "fs.azure.account.key.storekm01.dfs.core.windows.net",
    f'{key}'
)

In [0]:
display(dbutils.fs.ls("abfss://rawdata@storekm01.dfs.core.windows.net/"))

path,name,size,modificationTime
abfss://rawdata@storekm01.dfs.core.windows.net/2025-01-01/,2025-01-01/,0,1736093953000
abfss://rawdata@storekm01.dfs.core.windows.net/2025-01-02/,2025-01-02/,0,1736093973000
abfss://rawdata@storekm01.dfs.core.windows.net/2025-01-03/,2025-01-03/,0,1736094000000


In [0]:
dbutils.widgets.text('Date','')
input_value= dbutils.widgets.get('Date')

In [0]:
import pyspark
# Create a spark session using getOrCreate() function 
from pyspark.sql import SparkSession 
spark_session = SparkSession.builder.getOrCreate() 

In [0]:
df_category = spark_session.read.format( 
    "csv").option( 
    "header", True).load(f'{raw_folder_path}/product category.csv') 

In [0]:
df_category.show()

+-----------------+-----------------------+---------------+--------------------+--------------------+
|ProductCategoryID|ParentProductCategoryID|           Name|             rowguid|        ModifiedDate|
+-----------------+-----------------------+---------------+--------------------+--------------------+
|                1|                      1|          Bikes|cfbda25c-df71-47a...|2002-06-01T00:00:...|
|                2|                      2|     Components|c657828d-d808-4ab...|2002-06-01T00:00:...|
|                3|                      3|       Clothing|10a7c342-ca82-48d...|2002-06-01T00:00:...|
|                4|                      4|    Accessories|2be3be36-d9a2-4ee...|2002-06-01T00:00:...|
|                5|                      1| Mountain Bikes|2d364ade-264a-433...|2002-06-01T00:00:...|
|                6|                      1|     Road Bikes|000310c0-bcc8-42c...|2002-06-01T00:00:...|
|                7|                      1|  Touring Bikes|02c5061d-ecdc-427...|20

In [0]:
df_category=df_category.withColumnRenamed("Name","Category Name")

In [0]:
df_product = spark_session.read.format( 
    "csv").option( 
    "header", True).load(f'{raw_folder_path}/product.csv')

In [0]:
df_product.show()

+---------+--------------------+-------------+-----+------------+---------+----+-------+-----------------+--------------+--------------------+--------------------+----------------+--------------------+----------------------+--------------------+--------------------+
|ProductID|                Name|ProductNumber|Color|StandardCost|ListPrice|Size| Weight|ProductCategoryID|ProductModelID|       SellStartDate|         SellEndDate|DiscontinuedDate|      ThumbNailPhoto|ThumbnailPhotoFileName|             rowguid|        ModifiedDate|
+---------+--------------------+-------------+-----+------------+---------+----+-------+-----------------+--------------+--------------------+--------------------+----------------+--------------------+----------------------+--------------------+--------------------+
|      680|HL Road Frame - B...|   FR-R92B-58|Black|   1059.3100|1431.5000|  58|1016.04|               18|             6|2002-06-01T00:00:...|                NULL|            NULL|R0lGODlhUAAxAPcAA..

In [0]:
df_prodcat= df_product.join(df_category,df_product.ProductCategoryID== df_category.ProductCategoryID ).drop(df_product.ProductCategoryID)
df_prodcat.show()

+---------+--------------------+-------------+-----+------------+---------+----+-------+--------------+--------------------+--------------------+----------------+--------------------+----------------------+--------------------+--------------------+-----------------+-----------------------+-------------+--------------------+--------------------+
|ProductID|                Name|ProductNumber|Color|StandardCost|ListPrice|Size| Weight|ProductModelID|       SellStartDate|         SellEndDate|DiscontinuedDate|      ThumbNailPhoto|ThumbnailPhotoFileName|             rowguid|        ModifiedDate|ProductCategoryID|ParentProductCategoryID|Category Name|             rowguid|        ModifiedDate|
+---------+--------------------+-------------+-----+------------+---------+----+-------+--------------+--------------------+--------------------+----------------+--------------------+----------------------+--------------------+--------------------+-----------------+-----------------------+-------------+--

In [0]:
df_product_data= df_prodcat.select('ProductID','Name','StandardCost','ListPrice','Weight','Size','Category Name')
df_product_data.show()

+---------+--------------------+------------+---------+-------+----+-------------+
|ProductID|                Name|StandardCost|ListPrice| Weight|Size|Category Name|
+---------+--------------------+------------+---------+-------+----+-------------+
|      680|HL Road Frame - B...|   1059.3100|1431.5000|1016.04|  58|  Road Frames|
|      706|HL Road Frame - R...|   1059.3100|1431.5000|1016.04|  58|  Road Frames|
|      707|Sport-100 Helmet,...|     13.0863|  34.9900|   NULL|NULL|      Helmets|
|      708|Sport-100 Helmet,...|     13.0863|  34.9900|   NULL|NULL|      Helmets|
|      709|Mountain Bike Soc...|      3.3963|   9.5000|   NULL|   M|        Socks|
|      710|Mountain Bike Soc...|      3.3963|   9.5000|   NULL|   L|        Socks|
|      711|Sport-100 Helmet,...|     13.0863|  34.9900|   NULL|NULL|      Helmets|
|      712|        AWC Logo Cap|      6.9223|   8.9900|   NULL|NULL|         Caps|
|      713|Long-Sleeve Logo ...|     38.4923|  49.9900|   NULL|   S|      Jerseys|
|   

In [0]:
df_product_data=df_product_data.withColumnRenamed("Name","Product_Name")\
    .withColumnRenamed("Category Name","Category_Name")
df_product_data.show()

+---------+--------------------+------------+---------+-------+----+-------------+
|ProductID|        Product_Name|StandardCost|ListPrice| Weight|Size|Category_Name|
+---------+--------------------+------------+---------+-------+----+-------------+
|      680|HL Road Frame - B...|   1059.3100|1431.5000|1016.04|  58|  Road Frames|
|      706|HL Road Frame - R...|   1059.3100|1431.5000|1016.04|  58|  Road Frames|
|      707|Sport-100 Helmet,...|     13.0863|  34.9900|   NULL|NULL|      Helmets|
|      708|Sport-100 Helmet,...|     13.0863|  34.9900|   NULL|NULL|      Helmets|
|      709|Mountain Bike Soc...|      3.3963|   9.5000|   NULL|   M|        Socks|
|      710|Mountain Bike Soc...|      3.3963|   9.5000|   NULL|   L|        Socks|
|      711|Sport-100 Helmet,...|     13.0863|  34.9900|   NULL|NULL|      Helmets|
|      712|        AWC Logo Cap|      6.9223|   8.9900|   NULL|NULL|         Caps|
|      713|Long-Sleeve Logo ...|     38.4923|  49.9900|   NULL|   S|      Jerseys|
|   

In [0]:
df_product_data.printSchema()

root
 |-- ProductID: string (nullable = true)
 |-- Product_Name: string (nullable = true)
 |-- StandardCost: string (nullable = true)
 |-- ListPrice: string (nullable = true)
 |-- Weight: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Category_Name: string (nullable = true)



In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, StringType, DoubleType

df_product_data = (df_product_data
                   .withColumn("ProductID", col("ProductID").cast(IntegerType()))
                   .withColumn("Product_Name", col("Product_Name").cast(StringType()))
                   .withColumn("StandardCost", col("StandardCost").cast(DoubleType()))
                   .withColumn("ListPrice", col("ListPrice").cast(DoubleType()))
                   .withColumn("Weight", col("Weight").cast(DoubleType()))
                   .withColumn("Size", col("Size").cast(DoubleType()))
                   .withColumn("Category_Name", col("Category_Name").cast(StringType())))
 

In [0]:
df_product_data.printSchema()

root
 |-- ProductID: integer (nullable = true)
 |-- Product_Name: string (nullable = true)
 |-- StandardCost: double (nullable = true)
 |-- ListPrice: double (nullable = true)
 |-- Weight: double (nullable = true)
 |-- Size: double (nullable = true)
 |-- Category_Name: string (nullable = true)



In [0]:
product_data_processed= df_product_data.write.mode( "overwrite").option("header", "true").csv(f'{processed_folder_path}/productdata')